In [1]:
%matplotlib notebook
import os
import h5py
import covseisnet as cn
import obspy
import numpy as np
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import numpy.ma as ma
def insert_str(string, str_to_insert, index):
    return string[:index] + str_to_insert + string[index:]

In [2]:
def suppr_str(string, index, number_to_suppr):
    temp= index+number_to_suppr
    return string[:index] + string[temp:]
def suppr_gaps(stream):
    for i in range(len(stream)):
        average = stream[i].data.mean()
        stream[i].data=ma.filled(stream[i].data,average)
    return stream
def merge_traces_data_together(stream,number):
    for i in range(0,640*number,number):
        stream[int(i/number)]=stream[i]
        for j in range(1,number):
            stream[int(i/number)]+=stream[i+j]
    for i in range(640):
        for j in range(number-1):
                stream.pop(640)
    return stream

In [3]:
def read_stream(name_of_folder):
    
    hollow_path = r"C:\Users\MFE\ansel\\ID.00000_20210701.mseed"
    path = insert_str(hollow_path,name_of_folder,19)
    hollow_path1= suppr_str(path,42,1)
    hollow_path2= suppr_str(hollow_path1,42,1)
    hollow_path3= suppr_str(hollow_path2,42,1)
    stream = cn.arraystream.read(path)
    for i in range(9):
        temp_path = insert_str(hollow_path1,str(i+1),46)
        print(temp_path)
        stream += cn.arraystream.read(temp_path)
    for i in range(10,100):
        temp_path = insert_str(hollow_path2,str(i),45)
        stream += cn.arraystream.read(temp_path)
    for i in range(100,640):
        temp_path = insert_str(hollow_path3,str(i),44)
        stream += cn.arraystream.read(temp_path)
    return stream

In [4]:
stream1 =read_stream('20210701_T062219.81')
stream2 =read_stream('20210701_T062239.81')
stream3 =read_stream('20210701_T062259.81')
stream4 =read_stream('20210701_T062319.81')
stream5 =read_stream('20210701_T062339.81')
stream6 =read_stream('20210701_T062359.81')
stream7 =read_stream('20210701_T062419.81')
stream_count = 7
size=39*stream_count,len(stream1),2048
data_total=np.zeros(size,dtype=np.float32)
print(data_total.shape)
for i in range(len(stream1)):
    for j in range(39):
        data_total[j][i]=stream1[i][2048*j:2048*(j+1)]
        data_total[39+j][i]=stream2[i][2048*j:2048*(j+1)]
        data_total[2*39+j][i]=stream3[i][2048*j:2048*(j+1)]
        data_total[3*39+j][i]=stream4[i][2048*j:2048*(j+1)]
        data_total[4*39+j][i]=stream5[i][2048*j:2048*(j+1)]
        data_total[5*39+j][i]=stream6[i][2048*j:2048*(j+1)]
        data_total[6*39+j][i]=stream7[i][2048*j:2048*(j+1)]
# create an array containg the 3 streams ( used to register the reconstruction )
print(data_total[0][2][0:5])

C:\Users\MFE\ansel\20210701_T062219.81\ID.00001_20210701.mseed
C:\Users\MFE\ansel\20210701_T062219.81\ID.00002_20210701.mseed
C:\Users\MFE\ansel\20210701_T062219.81\ID.00003_20210701.mseed
C:\Users\MFE\ansel\20210701_T062219.81\ID.00004_20210701.mseed
C:\Users\MFE\ansel\20210701_T062219.81\ID.00005_20210701.mseed
C:\Users\MFE\ansel\20210701_T062219.81\ID.00006_20210701.mseed
C:\Users\MFE\ansel\20210701_T062219.81\ID.00007_20210701.mseed
C:\Users\MFE\ansel\20210701_T062219.81\ID.00008_20210701.mseed
C:\Users\MFE\ansel\20210701_T062219.81\ID.00009_20210701.mseed
C:\Users\MFE\ansel\20210701_T062239.81\ID.00001_20210701.mseed
C:\Users\MFE\ansel\20210701_T062239.81\ID.00002_20210701.mseed
C:\Users\MFE\ansel\20210701_T062239.81\ID.00003_20210701.mseed
C:\Users\MFE\ansel\20210701_T062239.81\ID.00004_20210701.mseed
C:\Users\MFE\ansel\20210701_T062239.81\ID.00005_20210701.mseed
C:\Users\MFE\ansel\20210701_T062239.81\ID.00006_20210701.mseed
C:\Users\MFE\ansel\20210701_T062239.81\ID.00007_2021070

In [5]:
for i in range(len(stream1)):
    for j in range(39*stream_count):
        data_total[j][i]=data_total[j][i]/data_total[j][i].std()

In [6]:
def save_arrays(X, save_dir):
    file_paths = []
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    for i in range(X.shape[0]):
        file_path = os.path.join(save_dir, f"array{i}.npy")
        np.save(file_path, X[i])
        file_paths.append(file_path)
    return file_paths

In [7]:
save_dir = r"C:\Users\MFE\ansel\J-Invariant encoder\Stored arrays_test"
listing = save_arrays(data_total,save_dir)


In [8]:
# Using the join method to write the entire list to a file
with open('my_list.txt', 'w') as f:
    f.write('\n'.join(map(str, listing)))
